# 评测chatgpt解决指代消解问题

In [ ]:
import os
from typing import List

from datasets import load_dataset



In [ ]:
wsc273_dataset = load_dataset('winograd_wsc', name='wsc273', split='test')
wsc273_dataset

In [ ]:
wsc273_dataset[0]

In [ ]:
!pip install openai==0.28.0

In [ ]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [ ]:
openai.__version__

In [25]:
def query_format_helper(pronoun: str, answers: List[str]) -> str:
    return f'Q: In the previous statement, does "{pronoun}" refer to {answers[0]} or {answers[1]}? A:'


def construct_query_from_schema(
        text: str, pronoun: str, answers: List[str]
) -> str:
    return f"S:{text} {query_format_helper(pronoun, answers)}"

c = 0
for example in wsc273_dataset:
    query = construct_query_from_schema(example['text'], example['pronoun'], example['options'])
    print(query)
    break

S:The city councilmen refused the demonstrators a permit because they feared violence. Q: In the previous statement, does "they" refer to The city councilmen or The demonstrators? A:


In [21]:
def get_openai_answer(query_prompt: str, add_leading: bool = True) -> str:
    init_prompt_starter = "The following are pairs of Winograd Schema in the form of a statement S, a question Q, and an answer A:"
    init_prompt1 = "S: The cat went through the door, but it's tail got stuck. Q: In the previous statement, what does 'it' refer to? A: The cat."
    init_prompt2 = "S: The cat tried to go through the door, but it was too small. Q: In the previous statement, what does 'it' refer to? A: The door."
    init_prompt3 = "S: Fedex made more profit than UPS last year, but that was mostly due to the success of the new delivery system they implemented. Q: In the previous statement, what does 'they' refer to? A: Fedex."
    init_prompt4 = "S: Sam tried to buy Xerxes lunch, but he wouldn't allow it. Q: In the previous statement, who does 'he' refer to? A: Xerxes."

    # add leading prompts to cue model...
    if add_leading:
        query_prompt = f"{init_prompt_starter} {init_prompt1}, {init_prompt2}, {init_prompt3}, {init_prompt4}, {query_prompt}"

    response = request_gpt_azure(query_prompt)
    return response

In [22]:
import openai


def request_gpt_azure(prompt):
    openai.api_type = "azure"
    openai.api_version = "2023-05-15"
    openai.api_base = "https://xxx"
    openai.api_key = "xxx"
    engine="gpt-35-turbo"
    # engine="gpt-4"

    rsp = openai.ChatCompletion.create(
            engine=engine,
            messages=[{"role": "user", "content": prompt}],
            request_timeout=120,
            temperature=0.5
    )
    return rsp['choices'][0]['message']['content']


prompt = '''
为啥我没出席我父母的婚礼
'''


# r = request_gpt_azure('为什么我没有被邀请参加父母的婚礼?', 'x')
r = request_gpt_azure(prompt)
print(r)

作为 AI，我无法确定您为什么没有出席您父母的婚礼。这可能涉及到您个人的情况和原因，例如您可能有其他计划或事情要处理，或者您可能与您的父母有不愉快的关系。无论原因是什么，如果您感到困惑或需要帮助，我建议您与您的家人或专业人士进行交流并寻求支持。


In [24]:
wsc273_dataset[0]

{'text': 'The city councilmen refused the demonstrators a permit because they feared violence.',
 'pronoun': 'they',
 'pronoun_loc': 63,
 'quote': 'they feared violence',
 'quote_loc': 63,
 'options': ['The city councilmen', 'The demonstrators'],
 'label': 0,
 'source': '(Winograd 1972)'}

In [29]:
c = 0
right_count = 0
for example in wsc273_dataset:
    query = construct_query_from_schema(example['text'], example['pronoun'], example['options'])
    print('Input:', query)
    r = get_openai_answer(query)
    print('Output:', r)
    truth = example['options'][example['label']]
    print('Truth:', truth)
    print('')

    if r and truth and truth.lower() in r.lower():
        right_count += 1
    c += 1
    if c > 100:
        print('acc:', right_count / c, ' count:', c, ' right count:', right_count)
        break

Input: S:The city councilmen refused the demonstrators a permit because they feared violence. Q: In the previous statement, does "they" refer to The city councilmen or The demonstrators? A:
Output: The city councilmen.
Truth: The city councilmen

Input: S:The city councilmen refused the demonstrators a permit because they advocated violence. Q: In the previous statement, does "they" refer to The city councilmen or The demonstrators? A:
Output: The demonstrators.
Truth: The demonstrators

Input: S:The trophy doesn't fit into the brown suitcase because it is too large. Q: In the previous statement, does "it" refer to the trophy or the suitcase? A:
Output: The suitcase.
Truth: the trophy

Input: S:The trophy doesn't fit into the brown suitcase because it is too small. Q: In the previous statement, does "it" refer to the trophy or the suitcase? A:
Output: The suitcase.
Truth: the suitcase

Input: S:Joan made sure to thank Susan for all the help she had recieved. Q: In the previous statemen

In [30]:
print('acc:', right_count / c, ' count:', c, ' right count:', right_count)

acc: 0.693069306930693  count: 101  right count: 70
